In [1]:
import numpy as np
import shutil
import os
from ultralytics import YOLO
import cv2
from tkinter import messagebox
import time
import PySide2

ModuleNotFoundError: No module named 'PySide2'

In [2]:
# model = YOLO("Weights\\Small (n)\\Trial 2\Cavs2003_best.pt")

# model.predict(r"C:\Users\Aseem Gill\Desktop\University\mini.mov",save_txt=True)

In [3]:
def yolo_text_to_dict(lines, classes):
    
    dic = {}
        
    items = {}
        
    for i in lines:
        i.strip()
        obj = i.split(" ")[0]
        x_coord = i.split(" ")[1]
        y_coord = i.split(" ")[2]
        width = i.split(" ")[3]
        height = i.split(" ")[4]
        
        
        items["X_coord"] = float(x_coord)
        items["Y_coord"] = float(y_coord)
        items["Width"] = float(width)
        items["Height"] = float(height)
        
        dic[classes[int(obj)]] = dict(items)
        
        
    return dic

In [4]:
# video_num = len(list(os.listdir("Video")))
video_num = 0

classes = ["LeBron", "Ball", "Net"]

video_dir = "Video/video_" + str(video_num + 1)
annot_dir = "Video/video_" + str(video_num + 1) + "/annots" 

if not os.path.exists(video_dir):
    os.mkdir(video_dir)
    
if not os.path.exists(annot_dir):
    os.mkdir(annot_dir)

frame_max = [] 

for i in list(os.listdir(annot_dir)):
        
    frame_max.append(int(i.split(".")[0].split("_")[1]))
    
frame_max = max(frame_max)

frames_info = {}

for i in range(1,frame_max+1):
        
    path = annot_dir + "\mini_" + str(i) + ".txt"
    
    if os.path.exists(path):
    
        with open(path) as f:

            lines = f.readlines()

        frames_info["Frame_" + str(i)] = yolo_text_to_dict(lines, classes)
        
    else:
        frames_info["Frame_" + str(i)] = {}

In [5]:
frames_info

{'Frame_1': {'Ball': {'X_coord': 0.855208,
   'Y_coord': 0.488032,
   'Width': 0.0229167,
   'Height': 0.0345745},
  'LeBron': {'X_coord': 0.0479167,
   'Y_coord': 0.444149,
   'Width': 0.0875,
   'Height': 0.218085},
  'Net': {'X_coord': 0.0385417,
   'Y_coord': 0.234043,
   'Width': 0.04375,
   'Height': 0.0638298}},
 'Frame_2': {'Ball': {'X_coord': 0.848958,
   'Y_coord': 0.496011,
   'Width': 0.0270833,
   'Height': 0.0345745},
  'LeBron': {'X_coord': 0.053125,
   'Y_coord': 0.453457,
   'Width': 0.0979167,
   'Height': 0.220745},
  'Net': {'X_coord': 0.0427083,
   'Y_coord': 0.235372,
   'Width': 0.04375,
   'Height': 0.0664894}},
 'Frame_3': {'Ball': {'X_coord': 0.845833,
   'Y_coord': 0.515957,
   'Width': 0.025,
   'Height': 0.0319149},
  'LeBron': {'X_coord': 0.0572917,
   'Y_coord': 0.457447,
   'Width': 0.110417,
   'Height': 0.218085},
  'Net': {'X_coord': 0.0479167,
   'Y_coord': 0.234043,
   'Width': 0.0416667,
   'Height': 0.0638298}},
 'Frame_4': {'LeBron': {'X_coord': 

In [6]:
def shot_check(curr_frame, waiting):
    
    if "Ball" in list(curr_frame.keys()) and "Net" in list(curr_frame.keys()) and waiting==0:

        ball = curr_frame["Ball"]
        net = curr_frame["Net"]
        
        ball_above_net = ball["Y_coord"] < (net["Y_coord"])
        
        ball_aligned_r = (ball["X_coord"] > (net["X_coord"] - 2*ball["Width"]))
        ball_aligned_l = (ball["X_coord"] < (net["X_coord"] + 2*ball["Width"]))        
        
        if  ball_above_net and ball_aligned_r and ball_aligned_l:
            return("Shot Up")
        
    elif "Ball" in list(curr_frame.keys()) and "Net" in list(curr_frame.keys()) and waiting > 0:
        
        ball = curr_frame["Ball"]
        net = curr_frame["Net"]
        
        ball_below_net = ball["Y_coord"] > (net["Y_coord"])
        
        ball_aligned_r = (ball["X_coord"] > (net["X_coord"] - 0.25*ball["Width"]))
        ball_aligned_l = (ball["X_coord"] < (net["X_coord"] + 0.25*ball["Width"]))        
        
        if  ball_below_net and ball_aligned_r and ball_aligned_l:
            return("+1 FG")
        
    return 0

In [7]:
i = 0


waiting = 0
status = 0
FG = 0
FGA = 0

cap = cv2.VideoCapture(r"C:\Users\Aseem Gill\Desktop\University\mini.mov")
ret, frame = cap.read()
while(1):
    ret, frame = cap.read()
    if cv2.waitKey(1) & 0xFF == ord('q') or ret==False :
        cap.release()
        cv2.destroyAllWindows()
        break
    
    i += 1
    
    curr_frame_info = frames_info["Frame_" + str(i)]
        
#     print(action)
    
    action = shot_check(curr_frame_info, waiting)
    
    if action:
        
        if action == "Shot Up" and status == 0:
            status = 1
        
        print("Status {} Waiting {}".format(status,waiting))
        
        if status:
            waiting += 1
            
        if waiting > 30:
            status = 0
#             cap.release()
#             cv2.destroyAllWindows()
#             break
            
        if action == "+1 FG":
            print(action)
            FG += 1
            FGA += 1
#             input("Hit Enter")
            
#             messagebox.showinfo(title="SCORE", message="LeBron Career Field Goals: {}/{}".format(FG,FGA))
#             cap.release()
#             cv2.destroyAllWindows()
#             break
            waiting = 0
            status = 0
            
#         input("Action Occurred")
            
    if status:
        waiting += 1

    if waiting > 30:
        status = 0
#         cap.release()
#         cv2.destroyAllWindows()
#         break
    
    time.sleep(0.03)
        

    
#     input("Press Enter to continue...")
    cv2.imshow("frame",frame)
    
#     print(i)
#         cv2.imwrite("more\\2003Cavs_frame" + str(i) + ".jpg",frame)

Status 1 Waiting 0
Shot Up
Status 1 Waiting 19
+1 FG
+1 FG
Status 1 Waiting 0
Shot Up
Status 1 Waiting 27
+1 FG
+1 FG
Status 1 Waiting 0
Shot Up
Status 1 Waiting 28
+1 FG
+1 FG


In [10]:
import sys

from PySide2.QtUiTools import QUiLoader
from PySide2.QtWidgets import QApplication, QMainWindow, QAction, QMessageBox, QFileDialog, QTextBrowser
from PySide2.QtCore import QFile


class MyMainWindow(QMainWindow):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.setWindowTitle("Demo QtWidget App")

    def closeEvent(self, event):
        msgBox = QMessageBox()
        msgBox.setWindowTitle("Quit?")
        msgBox.setText("Exit application?")
        msgBox.setStandardButtons(QMessageBox.Yes | QMessageBox.No)
        msgBox.setDefaultButton(QMessageBox.No)
        ret = msgBox.exec_()
        if ret == QMessageBox.Yes:
            event.accept()
        else:
            event.ignore()


if __name__ == '__main__':
    app = QApplication([])
    file = QFile("custom_widget_original.ui")
    #file = QFile("custom_widget_modified.ui")
    file.open(QFile.ReadOnly)
    loader = QUiLoader()
    loader.registerCustomWidget(MyMainWindow)
    main_window = loader.load(file)
    main_window.show()
    sys.exit(app.exec_())


ModuleNotFoundError: No module named 'PySide2'

In [ ]:
curr_frame = frames_info["Frame_195"]
print(curr_frame)
list(curr_frame.keys())

In [ ]:
bool = 0

In [13]:
cv2.imshow("frame",frame)
cv2.destroyWindow("frame")

In [56]:
with open(r"C:\Users\Aseem Gill\Desktop\University\Cavs2003\Video\video_1\annots\mini_329.txt") as f:
    
    lines = f.readlines()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Aseem Gill\\Desktop\\University\\Cavs2003\\Video\\video_1\\annots\\mini_329.txt'

In [57]:
lines

['0 0.544792 0.617021 0.897917 0.744681\n']

In [38]:
classes = ["LeBron", "Ball", "Net"]

# Function for Getting Dicts
#### object-class x y width height

In [20]:
with open(r"C:\Users\Aseem Gill\Desktop\University\Cavs2003\Video\video_1\annots\mini_516.txt") as f:
    
    lines = f.readlines()

In [36]:
yolo_text_to_dict(lines, classes)

['0 0.913542 0.5 0.172917 1\n', '0 0.933333 0.513298 0.133333 0.904255\n', '0 0.528125 0.462766 0.19375 0.590426\n']
0 0.913542 0.5 0.172917 1

0 0.933333 0.513298 0.133333 0.904255

0 0.528125 0.462766 0.19375 0.590426



{'LeBron': {'X_coord': 0.528125,
  'Y_coord': 0.462766,
  'Width': 0.19375,
  'Height': 0.590426}}

In [34]:
print(frame_max)

901


In [9]:
help(model.predict)

Help on method predict in module ultralytics.yolo.engine.model:

predict(source=None, stream=False, **kwargs) method of ultralytics.yolo.engine.model.YOLO instance
    Perform prediction using the YOLO model.
    
    Args:
        source (str | int | PIL | np.ndarray): The source of the image to make predictions on.
                      Accepts all source types accepted by the YOLO model.
        stream (bool): Whether to stream the predictions or not. Defaults to False.
        **kwargs : Additional keyword arguments passed to the predictor.
                   Check the 'configuration' section in the documentation for all available options.
    
    Returns:
        (List[ultralytics.yolo.engine.results.Results]): The prediction results.



In [4]:
import cv2

In [7]:
i = 0

model = YOLO("Cavs2003_best.pt")

cap = cv2.VideoCapture(r"C:\Users\Aseem Gill\Desktop\University\LeBron James First NBA Game, Full Highlights vs Kings (2003.10.29) - MUST WATCH Debut! HD.mp4")
ret, frame = cap.read()
while(1):
    ret, frame = cap.read()
    if cv2.waitKey(1) & 0xFF == ord('q') or ret==False :
        cap.release()
        cv2.destroyAllWindows()
        break
    labelled = model.predict(frame,show=True)
    
    labelled = np.array(labelled)
    
    print(labelled)
    
    cv2.imshow("frame",labelled)
#     i += 1
#     if i in frames:
#         cv2.imwrite("images\\2003Cavs_frame" + str(i) + ".jpg",frame)

WARNING  Environment does not support cv2.imshow() or PIL Image.show()

Ultralytics YOLOv8.0.39  Python-3.9.12 torch-1.13.1+cpu CPU
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs

0: 384x640 143.6ms
Speed: 1.0ms pre-process, 143.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


[]


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'
